##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Key classification with an RNN for Tensor Flow Lite

Classify USB data which comes in packets of 8 bytes representing shift and key presses.

## Setup

In [0]:
!pip install tensorflow
!pip install tensorflow-text
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text

## Setup input pipeline

Data for the pipeline needs to be in the form of a numpy array so we wrap our simple array in the call to np.array

Todo: Generate this data some how.

In [0]:
import numpy as np

dataset = np.array([ [2,0,4,0,0,0,0,0],[2,0,4,0,0,0,0,0],[0,0,5,0,0,0,0,0]])  # The letter A with a shift
resultset = np.array([[1,0],[1,0],[0,1]])

for ex in dataset:
  print(ex)

## Prepare the data for training

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 100

#Cheating
train_dataset= dataset
test_dataset = dataset

train_results = resultset
test_results = resultset

Lets take a look at one of these to see how it now looks 

In [0]:
print("Batch shape:", train_dataset.shape)
print("label shape:", train_results.shape)

## Create the model

Build a `tf.keras.Sequential` model

Use the input_shape parameter so that the convertion process can work correctly

A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input—and then to the next.

The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the output. This helps the RNN to learn long range dependencies.

In [0]:

model = tf.keras.Sequential([
  tf.keras.layers.Dense(16, activation='relu', input_shape=(8,)),
  tf.keras.layers.Dense(2)
  ])

model.summary()

The optimiser used for compiling seems to affect what we get when we put it onto the Arduino, for examples am getting zeros and overflows rather than data.

In [0]:
#model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.compile (optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#              optimizer=tf.keras.optimizers.Adam(1e-3),
#              metrics=['accuracy'])

## Train the model

Training on a reduced data by using .take(20) on the batches and reducing the validation steps to speed verification of the technique. Can use the whole set once we know the process will work.

In [0]:
history = model.fit(train_dataset,train_results, epochs=1,
                    validation_data=(test_dataset,test_results), 
                    validation_steps=1)

In [0]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [0]:
# Save the model
model.save("keyclassification_model") 

## Test the model

In [0]:
# predict on a sample data

res = model.predict(np.array([[2, 0, 4, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0],
                              [2, 0, 5, 6, 0, 0, 0, 0],
                              [0,0,5,0,0,0,0,0]]))
print(res)




# Export the model

Convert the model to TFLite then format as a big C array.

Based on https://github.com/eloquentarduino/tinymlgen/blob/master/tinymlgen/tinymlgen.py

Ref https://blog.tensorflow.org/2019/06/tensorflow-integer-quantization.html

In [0]:
!pip install hexdump

Did some experiments with the optimisations.

```
optimizers = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
```

results in 

```
Initialising...
Type FLOAT16 (10) not is not supported
Failed to initialize tensor 1
MicroAllocator: Failed to initialize.
AllocateTensors() failed
```

Tried also:

```
# From TinyML: Machine Learning with TensorFlow Lite on Arduino and Ultra-Low-Power
def representative_dataset_gen():
    for value in test_dataset:
        yield np.array(value,dtype=np.dtype((np.float32,8)),ndmin=2)

...

optimizers = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
```
but could not work out how to get a generator to produce data in the right way






In [0]:
#Experimenting with optimisations

import re
import hexdump
import tensorflow as tf

def port(model,optimize=True, variable_name='model_data',pretty_print=False):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    if optimize:
        if isinstance(optimize, bool):
            optimizers = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
        else:
            optimizers = optimize

        converter.optimizations = optimizers
    tflite_model = converter.convert()
    bytes = hexdump.dump(tflite_model).split(' ')
    c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
    c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
    if pretty_print:
        c = c.replace('{', '{\n\t').replace('}', '\n}')
        c = re.sub(r'(0x..?, ){12}', lambda x: '%s\n\t' % x.group(0), c)
    c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
    preamble = '''
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif
'''
    return preamble + c

In [0]:
c_code = port(model,optimize=True,pretty_print=True)

print(len(c_code))

File size needs to be < 400K to fit onto the device. Check the model_data_len value at the bottom of the file.
const int model_data_len = 109840 and a tiny bit of code comes to 90% of the availabe space.
But perhaps it also needs to be smaller than the available ram to be able to run? For example the sine model is just 2640 bytes;

In [0]:
c_file = open(r"text_model.h","w+")

n = c_file.write(c_code)
c_file.close()

# Testing the TFLite model

It is possible to reload the model back into the notebook and test it here.

Arena Size?

https://github.com/edgeimpulse/tflite-find-arena-size


Debugging TFLite
